In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_regression
import datetime
from sklearn.metrics import classification_report, confusion_matrix,recall_score , accuracy_score, precision_score, roc_auc_score

In [3]:
opera = pd.read_excel('/Users/angelique/Documents/GitHub/New_Opera_Company_Capstone/Excel and CSV Files/opera_4.xlsx')
opera.head()

,season,iso,city,composer,db,dd,nat,mf,work,worknat,...,performances_season_by_city,perf_per_1k_ppl_city_pop,opera_by_composer,performances_season_by_country_total,performances_season_by_city_total,perf_total_per_1k_city_pop,perf_total_per_10k_co_pop,Season Year,country_change_from_previous_season,city_change_from_previous_season
0,1213,al,Tirana,Lortzing,1801,1851,de,m,Ali Pascha von Janina,de,...,9,0.021506,Ali Pascha von Janina by Lor,110,110,0.262847,0.385301,2013-01-01,NaN,NaN
1,1213,al,Tirana,Mozart,1756,1791,at,m,Don Giovanni,it,...,9,0.021506,Don Giovanni by Moz,110,110,0.262847,0.385301,2013-01-01,NaN,NaN
2,1213,al,Tirana,Puccini,1858,1924,it,m,Tosca,it,...,9,0.021506,Tosca by Puc,110,110,0.262847,0.385301,2013-01-01,NaN,NaN
3,1213,am,Yerevan,Spendiaryan,1871,1928,am,m,Almast,am,...,5,0.004573,Almast by Spe,111,111,0.101510,0.371147,2013-01-01,NaN,NaN
4,1213,am,Yerevan,Tigranian,1879,1950,am,m,Anoush,am,...,5,0.004573,Anoush by Tig,111,111,0.101510,0.371147,2013-01-01,NaN,NaN


In [5]:
opera.columns

Index(['season', 'iso', 'city', 'composer', 'db', 'dd', 'nat', 'mf', 'work',
       'worknat', 'type', 'start date', 'performances', 'Country Name',
       'city population', 'country population', 'continent', 'sub-region',
       'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
       'performances_season_by_city', 'perf_per_1k_ppl_city_pop',
       'opera_by_composer', 'performances_season_by_country_total',
       'performances_season_by_city_total', 'perf_total_per_1k_city_pop',
       'perf_total_per_10k_co_pop', 'Season Year',
       'country_change_from_previous_season',
       'city_change_from_previous_season'],
      dtype='object')

In [7]:
opera.dtypes

season                                           int64
iso                                             object
city                                            object
composer                                        object
db                                              object
dd                                              object
nat                                             object
mf                                              object
work                                            object
worknat                                         object
type                                            object
start date                              datetime64[ns]
performances                                     int64
Country Name                                    object
city population                                  int64
country population                             float64
continent                                       object
sub-region                                      object
performanc

In [9]:
opera['Year'] = opera['start date'].dt.year
opera['Month'] = opera['start date'].dt.month
opera['Day'] = opera['start date'].dt.day
opera['Weekday'] = opera['start date'].dt.weekday
opera['Week_of_Year'] = opera['start date'].dt.isocalendar().week
opera['Days_Since_Start'] = (opera['start date'] - opera['start date'].min()).dt.days
opera.head()

,season,iso,city,composer,db,dd,nat,mf,work,worknat,...,perf_total_per_10k_co_pop,Season Year,country_change_from_previous_season,city_change_from_previous_season,Year,Month,Day,Weekday,Week_of_Year,Days_Since_Start
0,1213,al,Tirana,Lortzing,1801,1851,de,m,Ali Pascha von Janina,de,...,0.385301,2013-01-01,NaN,NaN,2013,3,23,5,12,273
1,1213,al,Tirana,Mozart,1756,1791,at,m,Don Giovanni,it,...,0.385301,2013-01-01,NaN,NaN,2013,5,18,5,20,329
2,1213,al,Tirana,Puccini,1858,1924,it,m,Tosca,it,...,0.385301,2013-01-01,NaN,NaN,2013,2,13,2,7,235
3,1213,am,Yerevan,Spendiaryan,1871,1928,am,m,Almast,am,...,0.371147,2013-01-01,NaN,NaN,2013,7,11,3,28,383
4,1213,am,Yerevan,Tigranian,1879,1950,am,m,Anoush,am,...,0.371147,2013-01-01,NaN,NaN,2013,5,11,5,19,322


In [11]:
opera.columns

Index(['season', 'iso', 'city', 'composer', 'db', 'dd', 'nat', 'mf', 'work',
       'worknat', 'type', 'start date', 'performances', 'Country Name',
       'city population', 'country population', 'continent', 'sub-region',
       'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
       'performances_season_by_city', 'perf_per_1k_ppl_city_pop',
       'opera_by_composer', 'performances_season_by_country_total',
       'performances_season_by_city_total', 'perf_total_per_1k_city_pop',
       'perf_total_per_10k_co_pop', 'Season Year',
       'country_change_from_previous_season',
       'city_change_from_previous_season', 'Year', 'Month', 'Day', 'Weekday',
       'Week_of_Year', 'Days_Since_Start'],
      dtype='object')

In [13]:
# Rank cities based on the sum of growth in performances
opera_city_growth = opera[['city', 'city_change_from_previous_season', 'iso']]

# Group by city and iso, and sum the 'city_change_from_previous_season' for each city
city_growth_sum = opera_city_growth.groupby(['city', 'iso'], as_index=False)['city_change_from_previous_season'].sum()

# Sort cities by total growth (sum), highest growth first
city_growth_sorted = city_growth_sum.sort_values(by='city_change_from_previous_season', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
opera_city_growth_unique = city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get the top cities with the highest total growth
top_cities = opera_city_growth_unique.head(25)  # Top 10 cities with the highest total growth

# Print the result
print(top_cities)

               city iso  city_change_from_previous_season
748          Moscow  ru                           27911.0
1216     Washington  us                           16128.0
1062  St Petersburg  ru                            9681.0
51        Arlington  us                            9648.0
263        Columbus  us                            3549.0
759         München  de                            3381.0
931        Richmond  us                            3042.0
846           Paris  fr                            2971.0
868    Philadelphia  us                            2916.0
327      Dusseldorf  de                            1677.0
315         Dresden  de                            1674.0
892     Portland OR  us                            1539.0
62          Atlanta  us                            1312.0
295          Denver  us                            1168.0
371         Firenze  it                            1163.0
1204       Voronezh  ru                             979.0
290          D

In [15]:
# Rank countries based on the sum of growth in performances
opera_country_growth = opera[['Country Name', 'country_change_from_previous_season', 'iso']]

# Group by country and iso, and sum the 'country_change_from_previous_season' for each country
country_growth_sum = opera_country_growth.groupby(['Country Name', 'iso'], as_index=False)['country_change_from_previous_season'].sum()

# Sort countries by total growth (sum), highest growth first
country_growth_sorted = country_growth_sum.sort_values(by='country_change_from_previous_season', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
opera_country_growth_unique = country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get the top cities with the highest total growth
top_countries = opera_country_growth_unique.head(25)  # Top 10 cities with the highest total growth

# Print the result
print(top_countries)

          Country Name iso  country_change_from_previous_season
58  Russian Federation  ru                             194425.0
34               Italy  it                              87941.0
65              Sweden  se                               6459.0
55              Poland  pl                               5653.0
64               Spain  es                               3115.0
50         Netherlands  nl                               2892.0
35               Japan  jp                                631.0
39              Latvia  lv                                529.0
23             Estonia  ee                                517.0
62            Slovenia  si                                509.0
6              Belarus  by                                450.0
73          Uzbekistan  uz                                450.0
2              Armenia  am                                442.0
13               China  cn                                396.0
59              Serbia  rs              

In [17]:
nan_count_per_column = opera.isna().sum()
print(nan_count_per_column)

season                                     0
iso                                        0
city                                       0
composer                                   5
db                                       227
dd                                      3593
nat                                       11
mf                                         0
work                                       5
worknat                                 1548
type                                       0
start date                                 0
performances                               0
Country Name                               0
city population                            0
country population                         0
continent                                  0
sub-region                                 0
performances_season_by_country             0
perf_per_10k_ppl_co_pop                    0
performances_season_by_city                0
perf_per_1k_ppl_city_pop                   0
opera_by_c

In [19]:
# Step 1: Select only numeric columns
numeric_columns = opera.select_dtypes(include=[np.number])

# Step 2: Check for infinity values in the numeric columns
infinity_counts = np.isinf(numeric_columns).sum()

# Step 3: Print the count of infinity values per numeric column
print("Infinity counts per numeric column:")
print(infinity_counts)

Infinity counts per numeric column:
season                                  0
performances                            0
city population                         0
country population                      0
performances_season_by_country          0
perf_per_10k_ppl_co_pop                 0
performances_season_by_city             0
perf_per_1k_ppl_city_pop                0
performances_season_by_country_total    0
performances_season_by_city_total       0
perf_total_per_1k_city_pop              0
perf_total_per_10k_co_pop               0
country_change_from_previous_season     0
city_change_from_previous_season        0
Year                                    0
Month                                   0
Day                                     0
Weekday                                 0
Week_of_Year                            0
Days_Since_Start                        0
dtype: Int64


In [21]:
# Define the condition for selecting rows
condition = (opera['iso'] == 'cu') & (opera['city'] == 'La Habana')

# Apply fillna() to the 'country_change_from_previous_season' column for the selected rows
opera.loc[condition, 'country_change_from_previous_season'] = opera.loc[
    condition, 'country_change_from_previous_season'
].fillna(2)

In [23]:
# Display rows where 'iso' is 'cu' and 'city' is 'La Habana'
print(opera[(opera['iso'] == 'cu') & (opera['city'] == 'La Habana')])

       season iso       city composer    db    dd nat mf          work  \
27083    1617  cu  La Habana   Mozart  1756  1791  at  m  Don Giovanni   

      worknat  ... perf_total_per_10k_co_pop Season Year  \
27083      it  ...                  0.001783  2017-01-01   

       country_change_from_previous_season city_change_from_previous_season  \
27083                                  2.0                              NaN   

       Year  Month Day Weekday  Week_of_Year  Days_Since_Start  
27083  2016     12  17       5            50              1638  

[1 rows x 36 columns]


In [25]:
# Step 1: Identify rows with missing values in 'country_change_from_previous_season'
country_missing = opera[opera['country_change_from_previous_season'].isna()]

# Step 2: Use rows without missing values to train the model
country_not_missing = opera.dropna(subset=['country_change_from_previous_season'])

# Separate features (X) and target (y) for rows without missing values
X_no_missing = country_not_missing[['city population', 'country population', 'performances_season_by_country', 
                                    'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 
                                    'Year']]
y_no_missing = country_not_missing['country_change_from_previous_season']

# Step 3: Train the Linear Regression Model for each iso
predicted_values = []

# Group the data by 'iso' and perform the prediction for each group
for iso_code, group in country_not_missing.groupby('iso'):
    # Separate the group into features (X) and target (y)
    X_group = group[['city population', 'country population', 'performances_season_by_country', 
                     'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year']]
    y_group = group['country_change_from_previous_season']
    
    # Train the linear regression model for this specific iso code
    model = LinearRegression()
    model.fit(X_group, y_group)
    
    # Get the rows for this iso code that have missing values
    missing_group = country_missing[country_missing['iso'] == iso_code]
    
    # If there are any missing values for this iso, predict them
    if not missing_group.empty:
        X_missing = missing_group[['city population', 'country population', 'performances_season_by_country', 
                                  'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year']]
        
        # Predict the missing values
        predictions = model.predict(X_missing)
        
        # Store the predicted values and add to the list
        predicted_values.extend(predictions)

# Step 4: Check the lengths to confirm they match
print(f"Length of country_missing: {len(country_missing)}")
print(f"Length of predicted values: {len(predicted_values)}")


Length of country_missing: 6558
Length of predicted values: 6558


In [27]:
# Step 5: Assign the predicted values to the original DataFrame
country_missing['country_change_from_previous_season'] = predicted_values

# Print the updated dataframe with predicted values
print(country_missing)

       season iso     city        composer    db    dd nat mf  \
0        1213  al   Tirana        Lortzing  1801  1851  de  m   
1        1213  al   Tirana          Mozart  1756  1791  at  m   
2        1213  al   Tirana         Puccini  1858  1924  it  m   
3        1213  am  Yerevan     Spendiaryan  1871  1928  am  m   
4        1213  am  Yerevan       Tigranian  1879  1950  am  m   
...       ...  ..      ...             ...   ...   ...  .. ..   
10091    1314  kz   Astana           Verdi  1813  1901  it  m   
10240    1314  mo    Macau          Mozart  1756  1791  at  m   
10241    1314  mo    Macau           Verdi  1813  1901  it  m   
10242    1314  mo    Macau           Verdi  1813  1901  it  m   
10243    1314  mo    Macau  Wagner,Richard  1813  1883  de  m   

                        work worknat  ... perf_total_per_10k_co_pop  \
0      Ali Pascha von Janina      de  ...                  0.385301   
1               Don Giovanni      it  ...                  0.385301   
2     

/var/folders/v8/j1kjtws546j8drwljwyklgrm0000gn/T/ipykernel_87521/1160055947.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_missing['country_change_from_previous_season'] = predicted_values


In [29]:
# Step 5: Fill the missing values with the predicted values
opera.loc[opera['country_change_from_previous_season'].isna(), 'country_change_from_previous_season'] = predicted_values

# Now 'df' has the missing values in 'country_change_from_previous_season' filled
print(opera.head())

   season iso     city     composer    db    dd nat mf                   work  \
0    1213  al   Tirana     Lortzing  1801  1851  de  m  Ali Pascha von Janina   
1    1213  al   Tirana       Mozart  1756  1791  at  m           Don Giovanni   
2    1213  al   Tirana      Puccini  1858  1924  it  m                  Tosca   
3    1213  am  Yerevan  Spendiaryan  1871  1928  am  m                 Almast   
4    1213  am  Yerevan    Tigranian  1879  1950  am  m                 Anoush   

  worknat  ... perf_total_per_10k_co_pop Season Year  \
0      de  ...                  0.385301  2013-01-01   
1      it  ...                  0.385301  2013-01-01   
2      it  ...                  0.385301  2013-01-01   
3      am  ...                  0.371147  2013-01-01   
4      am  ...                  0.371147  2013-01-01   

   country_change_from_previous_season city_change_from_previous_season  Year  \
0                             2.230995                              NaN  2013   
1             

In [31]:
nan_count_per_column = opera.isna().sum()
print(nan_count_per_column)

season                                     0
iso                                        0
city                                       0
composer                                   5
db                                       227
dd                                      3593
nat                                       11
mf                                         0
work                                       5
worknat                                 1548
type                                       0
start date                                 0
performances                               0
Country Name                               0
city population                            0
country population                         0
continent                                  0
sub-region                                 0
performances_season_by_country             0
perf_per_10k_ppl_co_pop                    0
performances_season_by_city                0
perf_per_1k_ppl_city_pop                   0
opera_by_c

In [33]:
# Define the condition for selecting rows
condition = (opera['iso'] == 'cu') & (opera['city'] == 'La Habana')

# Apply fillna() to the 'country_change_from_previous_season' column for the selected rows
opera.loc[condition, 'city_change_from_previous_season'] = opera.loc[
    condition, 'city_change_from_previous_season'
].fillna(2)

In [35]:
# Step 1: Identify rows with missing values in 'city_change_from_previous_season'
city_missing = opera[opera['city_change_from_previous_season'].isna()]

# Step 2: Use rows without missing values to train the model
city_not_missing = opera.dropna(subset=['city_change_from_previous_season'])

# Separate features (X) and target (y) for rows without missing values
X_no_missing = city_not_missing[['city population', 'country population', 'performances_season_by_country', 
                                    'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 
                                    'Year']]
y_no_missing = city_not_missing['city_change_from_previous_season']

# Step 3: Train the Linear Regression Model for each iso
predicted_values = []

# Group the data by 'iso' and perform the prediction for each group
for iso_code, group in city_not_missing.groupby('iso'):
    # Separate the group into features (X) and target (y)
    X_group = group[['city population', 'country population', 'performances_season_by_country', 
                     'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year']]
    y_group = group['city_change_from_previous_season']
    
    # Train the linear regression model for this specific iso code
    model = LinearRegression()
    model.fit(X_group, y_group)
    
    # Get the rows for this iso code that have missing values
    missing_group = city_missing[city_missing['iso'] == iso_code]
    
    # If there are any missing values for this iso, predict them
    if not missing_group.empty:
        X_missing = missing_group[['city population', 'country population', 'performances_season_by_country', 
                                  'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year']]
        
        # Predict the missing values
        predictions = model.predict(X_missing)
        
        # Store the predicted values and add to the list
        predicted_values.extend(predictions)

# Step 4: Check the lengths to confirm they match
print(f"Length of city_missing: {len(city_missing)}")
print(f"Length of predicted values: {len(predicted_values)}")

Length of city_missing: 7190
Length of predicted values: 7190


In [37]:
# Step 5: Fill the missing values with the predicted values
opera.loc[opera['city_change_from_previous_season'].isna(), 'city_change_from_previous_season'] = predicted_values

# Now 'df' has the missing values in 'country_change_from_previous_season' filled
print(opera.head())

   season iso     city     composer    db    dd nat mf                   work  \
0    1213  al   Tirana     Lortzing  1801  1851  de  m  Ali Pascha von Janina   
1    1213  al   Tirana       Mozart  1756  1791  at  m           Don Giovanni   
2    1213  al   Tirana      Puccini  1858  1924  it  m                  Tosca   
3    1213  am  Yerevan  Spendiaryan  1871  1928  am  m                 Almast   
4    1213  am  Yerevan    Tigranian  1879  1950  am  m                 Anoush   

  worknat  ... perf_total_per_10k_co_pop Season Year  \
0      de  ...                  0.385301  2013-01-01   
1      it  ...                  0.385301  2013-01-01   
2      it  ...                  0.385301  2013-01-01   
3      am  ...                  0.371147  2013-01-01   
4      am  ...                  0.371147  2013-01-01   

   country_change_from_previous_season city_change_from_previous_season  Year  \
0                             2.230995                         2.230995  2013   
1             

In [39]:
nan_count_per_column = opera.isna().sum()
print(nan_count_per_column)

season                                     0
iso                                        0
city                                       0
composer                                   5
db                                       227
dd                                      3593
nat                                       11
mf                                         0
work                                       5
worknat                                 1548
type                                       0
start date                                 0
performances                               0
Country Name                               0
city population                            0
country population                         0
continent                                  0
sub-region                                 0
performances_season_by_country             0
perf_per_10k_ppl_co_pop                    0
performances_season_by_city                0
perf_per_1k_ppl_city_pop                   0
opera_by_c

In [41]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Evaluate the model using mean absolute error, mean standard error, root mean square error and the rsquared score under lin_evaluate_model fuction
def lin_evaluate_model(y_test, y_pred):
    print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
    print(f"MSE: {mean_squared_error(y_test, y_pred)}")
    print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False)}")
    print(f"R2_Score: {r2_score(y_test, y_pred)}")

In [43]:
#Create a function for the more complex looped linear regressions
def evaluate_linear_regression(model, X, y, sub_region_groups, scaler):
    """
    Trains and evaluates a linear regression model for each sub-region,
    then returns the average MAE, RMSE, and R² score.

    Parameters:
    - model: Linear regression model.
    - X: Feature DataFrame.
    - y: Target variable DataFrame.
    - country_groups: Grouped data by sub-region.
    - scaler: StandardScaler instance.

    Returns:
    - Dictionary with average MAE, RMSE, and R² score.
    """
    mae_scores, rmse_scores, r2_scores = [], [], []

    for country_or_city, group in sub_region_groups:
        if len(group) > 1:
            # Extract features and target
            X_group = group[X.columns]
            y_group = group[y.name]

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X_group, y_group, test_size=0.2, random_state=42)

            # Scale the data
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # Train the model
            model.fit(X_train_scaled, y_train)

            # Make predictions
            predictions = model.predict(X_test_scaled)

            # Compute evaluation metrics
            mae_scores.append(mean_absolute_error(y_test, predictions))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test, predictions)))
            r2_scores.append(r2_score(y_test, predictions))

    # Compute average metrics
    avg_metrics = {
        'MAE': np.mean(mae_scores) if mae_scores else 0,
        'RMSE': np.mean(rmse_scores) if rmse_scores else 0,
        'R2': np.mean(r2_scores) if r2_scores else 0
    }

    return avg_metrics

In [45]:
final_results = []

Linear Regression models - I ran these first without grouping by sub-region to help the predictive model just as a starting point to see how a linear regression model would perform

In [48]:
X_country = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at country level)
y_country = opera['country_change_from_previous_season']

# Train-test split
X_train, X_test, y_train_country, y_test_country = train_test_split(X_country, y_country, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

# Model for country growth prediction
lin_country = LinearRegression()

# Train the model on scaled data
lin_country.fit(X_train_scaled, y_train_country)

# Transform the entire country data to scaled values before prediction (using the same scaler)
X_country_scaled = scaler.transform(X_country)

# Predict growth for the entire country dataset
opera['predicted_country_growth'] = lin_country.predict(X_country_scaled)

# Sort countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted = df_opera_country_growth_unique.head(10)
print(top_countries_predicted)

      Country Name  predicted_country_growth
263      Australia                 75.180739
190        Austria                 72.237065
2427       Finland                 63.241543
3883       Romania                 59.168818
4515      Slovenia                 58.507863
16338      Hungary                 58.266885
3390     Lithuania                 58.138222
3798        Poland                 56.789262
4615        Turkey                 47.124328
14132  Switzerland                 46.597059


In [50]:
# Make a prediction
y_pred_country_lin = lin_country.predict(X_test_scaled)

# Evaluate model
lin_evaluate_model(y_test_country, y_pred_country_lin)

MAE: 147.3972739647199
MSE: 41254.4963747648
RMSE: 203.1120291237444
R2_Score: 0.038655174243710566


In [52]:
# Independent variables (predictors)
X_city = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Train-test split
X_train, X_test, y_train_city, y_test_city = train_test_split(X_city, y_city, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

# Model for city growth prediction
lin_city = LinearRegression()

# Train the model on scaled data
lin_city.fit(X_train_scaled, y_train_city)

# Transform the entire city data to scaled values before prediction (using the same scaler)
X_city_scaled = scaler.transform(X_city)

# Predict growth for the entire city dataset
opera['predicted_city_growth'] = lin_country.predict(X_city_scaled)

# Sort cities based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 25 cities with highest predicted growth
top_cities_predicted = df_opera_city_growth_unique.head(10)
print(top_cities_predicted)

      iso         city  predicted_city_growth
263    au       Sydney              75.180739
190    at       Vienna              72.237065
2427   fi     Helsinki              63.241543
3883   ro  Cluj-Napoca              59.168818
4515   si      Maribor              58.507863
16338  hu     Budapest              58.266885
3390   lt      Vilnius              58.138222
3798   pl      Wroclaw              56.789262
10736  ro    Timisoara              55.113396
10113  lt       Kaunas              53.492196


In [54]:
# Make a prediction
y_pred_city_lin = lin_city.predict(X_test_scaled)

# Evaluate model
lin_evaluate_model(y_test_city, y_pred_city_lin)

MAE: 20.428280777427656
MSE: 5380.81251451332
RMSE: 73.35402180189796
R2_Score: 0.0036593842846965874


Linear Regression and grouping by sub-region for predictive analysis. This time I grouped the opera data by sub-regions before peforming the linear regression to see if it improved predictions.  The scores were significantly better for the cities prediction but they still were underperforming for the country data. 

In [57]:
# Independent variables (predictors)
X_country = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at country level)
y_country = opera['country_change_from_previous_season']

# Group by sub-region
sub_region_groups = opera.groupby('sub-region')

# Initialize the StandardScaler
scaler = StandardScaler()

# Model for country growth prediction
lin_country = LinearRegression()

# Store the predicted values in a DataFrame
predicted_growth_values = pd.Series(index=opera.index)  # Create an empty Series to store predictions

# Loop through each sub-region group to train and predict separately
for country_or_city, group in sub_region_groups:
    # If the group has more than one sample, split and train
    if len(group) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['country_change_from_previous_season']

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # Train the model on the training data
        lin_country.fit(X_train_scaled, y_train_country)

        # Scale the test data using the same scaler
        X_test_scaled = scaler.transform(X_test)

        # Make predictions for the test set
        predictions = lin_country.predict(X_test_scaled)

        # Assign predictions to the correct indices in the original DataFrame
        predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
    else:
        # If the group has only one sample, predict directly and append
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['country_change_from_previous_season']
        
        # Scale the data for this group
        X_scaled = scaler.fit_transform(X_group)

        # Train the model
        lin_country.fit(X_scaled, y_group)

        # Make predictions for this group
        predictions = lin_country.predict(X_scaled)
        
        # Assign predictions to the correct indices in the original DataFrame
        predicted_growth_values.loc[group.index] = predictions  # Correctly assign predictions to the original rows

# Assign the predicted values to the original DataFrame
opera['predicted_country_growth'] = predicted_growth_values

# Sort countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted = df_opera_country_growth_unique.head(10)
print(top_countries_predicted_lin)

             Country Name  predicted_country_growth
12366       United States                384.528795
16879          Kazakhstan                353.238890
10028          Kyrgyzstan                325.570095
32851          Uzbekistan                270.377483
30634  Russian Federation                259.203609
33363              Canada                248.547297
17247              Poland                180.091249
3100                Italy                 92.900284
15701               Egypt                 90.607635
3917              Romania                 77.953992


In [59]:
lin_metrics_country = evaluate_linear_regression(lin_country, X_country, y_country, sub_region_groups, scaler)

print("\n🔹 Average Linear Regression Metrics Across Sub-Regions for Countries:")
for metric, value in lin_metrics_country.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Linear Regression Metrics Across Sub-Regions for Countries:
   MAE: 62.5307
   RMSE: 103.0777
   R2: -1.5042


In [61]:
# Independent variables (predictors)
X_city = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Still group the cities by sub-region
sub_region_groups = opera.groupby('sub-region')

# Initialize the StandardScaler
scaler = StandardScaler()

# Model for country growth prediction
lin_city = LinearRegression()

# Store the predicted values in a DataFrame
predicted_growth_values = pd.Series(index=opera.index)  # Create an empty Series to store predictions

# Loop through each country group to train and predict separately
for country_or_city, group in sub_region_groups:
    # If the group has more than one sample, split and train
    if len(group) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # Train the model on the training data
        lin_city.fit(X_train_scaled, y_train_city)

        # Scale the test data using the same scaler
        X_test_scaled = scaler.transform(X_test)

        # Make predictions for the test set
        predictions = lin_city.predict(X_test_scaled)

        # Assign predictions to the correct indices in the original DataFrame
        predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
    else:
        # If the group has only one sample, predict directly and append
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']
        
        # Scale the data for this group
        X_scaled = scaler.fit_transform(X_group)

        # Train the model
        lin_city.fit(X_scaled, y_group)

        # Make predictions for this group
        predictions = lin_city.predict(X_scaled)
        
        # Assign predictions to the correct indices in the original DataFrame
        predicted_growth_values.loc[group.index] = predictions  # Correctly assign predictions to the original rows

# Assign the predicted values to the original DataFrame
opera['predicted_city_growth'] = predicted_growth_values

# Sort countries based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 countries with highest predicted growth
top_cities_predicted_lr1 = df_opera_city_growth_unique.head(10)
print(top_cities_predicted_lin)

      iso           city  predicted_city_growth
15701  eg     Alexandria              71.954844
30467  ru         Moscow              52.171187
26239  us     Washington              51.766670
10389  nz     Berhampore              42.547934
13054  us    Saint Louis              41.165920
11188  ru  St Petersburg              38.523518
11980  us      Arlington              34.697543
6536   am        Yerevan              34.630684
12661  us        Madison              32.847327
12412  us        Houston              32.374409


In [63]:
lin_metrics_city = evaluate_linear_regression(lin_city, X_city, y_city, sub_region_groups, scaler)

print("\n🔹 Average Linear Regression Metrics Across Sub-Regions for Cities:")
for metric, value in lin_metrics_city.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Linear Regression Metrics Across Sub-Regions for Cities:
   MAE: 12.7954
   RMSE: 19.0671
   R2: -4.0879


Logistic Regression Models - let's see if we can get a better predictive model using Logistic regression without countries being grouped first and then try it with grouping countries. In order to run a Logistic regression model, I binned the country_change_from_previous_season by quartiles into categories of Low, Medium, High and Very High so there is a category that is being used as the dependent outcome.

In [66]:
# Using this function to evaluate performance of models
def evaluate_model_performance(model, X, y, sub_region_groups, scaler, bins, labels):
    """
    Trains and evaluates a model for each sub-region, then returns the average accuracy, precision, and recall.

    Parameters:
    - model: The machine learning model to train (e.g., RandomForestClassifier, LogisticRegression).
    - X: Feature DataFrame.
    - y: Target variable DataFrame.
    - country_groups: Grouped data by sub-region.
    - scaler: StandardScaler instance.
    - bins: Bins for categorizing target values.
    - labels: Labels corresponding to bins.

    Returns:
    - Dictionary with average accuracy, precision, and recall.
    """
    accuracies, precisions, recalls = [], [], []

    for sub_region, group in sub_region_groups:
        if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
            # Extract features and target
            X_group = group[X.columns]
            y_group = group[y.name]

            # Convert the target variable to categorical labels
            y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

            # Scale the training and test data
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # Train only if there are multiple classes
            if len(y_train.unique()) > 1:
                model.fit(X_train_scaled, y_train)
                predictions = model.predict(X_test_scaled)

                # Store evaluation metrics
                accuracies.append(accuracy_score(y_test, predictions))
                precisions.append(precision_score(y_test, predictions, average='weighted', zero_division=0))
                recalls.append(recall_score(y_test, predictions, average='weighted', zero_division=0))

    # Compute the average metrics
    avg_metrics = {
        'accuracy': np.mean(accuracies) if accuracies else 0,
        'precision': np.mean(precisions) if precisions else 0,
        'recall': np.mean(recalls) if recalls else 0
    }

    return avg_metrics

In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Independent variables (predictors)
X_country = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at country level)
y_country = opera['country_change_from_previous_season']

# Convert continuous growth to categorical labels (Low, Medium, High)
q1 = opera['country_change_from_previous_season'].quantile(0.25)
q2 = opera['country_change_from_previous_season'].quantile(0.50)
q3 = opera['country_change_from_previous_season'].quantile(0.75)

bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

y_country_classified = pd.cut(y_country, bins=bins, labels=labels)

# Train-test split
X_train, X_test, y_train_country, y_test_country = train_test_split(X_country, y_country_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [70]:
# Initialize LogisticRegression
lr_country = LogisticRegression(max_iter=10000, multi_class='ovr')

# Define the hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'solver': ['liblinear', 'saga'],  # Use 'liblinear' and 'saga' or 'lbfgs' for L1
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# GridSearchCV with custom scoring
grid_search = GridSearchCV(estimator=lr_country, param_grid=param_grid, scoring=custom_scorer, cv=5)
grid_search.fit(X_train_scaled, y_train_country)

# Output the best parameters and the best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Best Parameters: {'C': 10, 'solver': 'saga'}
Best F1 Score: 0.48709776078392775


In [72]:
# Best parameters found during the grid search
best_params = grid_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_country = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=10000,
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Group the data by 'Country Name'
sub_region_groups = opera.groupby('sub-region')

# Loop through each country group to train and predict separately
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['country_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_country.unique()) > 1:
            # Train the model on the training data
            lr_country.fit(X_train_scaled, y_train_country)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_country.predict(X_test_scaled)

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping the sub-region: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping the sub-region: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_country_growth'] = predicted_growth_values

# Sort and get the top countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted_lr1 = df_opera_country_growth_unique.head(10)
print(top_countries_predicted_lr1)

Skipping the sub-region: Northern Africa due to insufficient class variation in the target.
Skipping the sub-region: South-eastern Asia due to insufficient class variation in the target.
             Country Name predicted_country_growth
14846             Germany                Very High
13313       United States                Very High
35890               Italy                Very High
30564  Russian Federation                Very High
11596      United Kingdom                Very High
9024                Spain                Very High
18084              Sweden                Very High
35573             Hungary                   Medium
27086      Czech Republic                   Medium
15895              France                   Medium


In [78]:
lr_metrics_country1 = evaluate_model_performance(lr_country, X_country, y_country, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Logistic Regression Metrics Across Sub-Regions for Countries using GridSearch:")
for metric, value in lr_metrics_country1.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Logistic Regression Metrics Across Sub-Regions for Countries using GridSearch:
   accuracy: 0.7206
   precision: 0.7168
   recall: 0.7206


In [80]:
# Independent variables (predictors)
X_city = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Convert continuous growth to categorical labels (Low, Medium, High)
q1 = opera['city_change_from_previous_season'].quantile(0.25)
q2 = opera['city_change_from_previous_season'].quantile(0.50)
q3 = opera['city_change_from_previous_season'].quantile(0.75)

bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

y_city_classified = pd.cut(y_city, bins=bins, labels=labels)

# Train-test split
X_train, X_test, y_train_city, y_test_city = train_test_split(X_city, y_city_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [82]:
# Initialize LogisticRegression
lr_city = LogisticRegression(max_iter=10000, multi_class='ovr')

# Define the hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'solver': ['liblinear', 'saga'],  # Use 'liblinear' and 'saga' or 'lbfgs' for L1
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# GridSearchCV with custom scoring
grid_search = GridSearchCV(estimator=lr_city, param_grid=param_grid, scoring=custom_scorer, cv=5)
grid_search.fit(X_train_scaled, y_train_city)

# Output the best parameters and the best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Best Parameters: {'C': 10, 'solver': 'liblinear'}
Best F1 Score: 0.3248013022646522


In [84]:
# Best parameters found during the grid search
best_params = grid_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_city = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=10000,
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region = opera.groupby('sub-region')

# Loop through each sub-region group to train and predict separately for cities
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this sub-region group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_city.unique()) > 1:
            # Train the model on the training data
            lr_city.fit(X_train_scaled, y_train_city)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_city.predict(X_test_scaled)

            # Ensure the indices of predictions and X_test match
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping sub-region: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping sub-region: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_city_growth'] = predicted_growth_values

# Sort and get the top cities based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 cities with highest predicted growth
top_cities_predicted_lr2 = df_opera_city_growth_unique.head(10)
print(top_cities_predicted_lr1)

      iso          city predicted_city_growth
38935  uz      Tashkent             Very High
32944  at          Graz             Very High
33004  at      Salzburg             Very High
32984  at          Linz             Very High
32959  at     Innsbruck             Very High
13343  us    Washington             Very High
32898  ar  Buenos Aires             Very High
32868  za     Cape Town             Very High
32839  us    Wilmington             Very High
33126  au     Melbourne             Very High


In [86]:
lr_metrics_city1 = evaluate_model_performance(lr_city, X_city, y_city, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Logistic Regression Metrics Across Sub-Regions for Cities using GridSearch:")
for metric, value in lr_metrics_city1.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Logistic Regression Metrics Across Sub-Regions for Cities using GridSearch:
   accuracy: 0.5203
   precision: 0.5411
   recall: 0.5203


Let's see if there's better results using RandomizedSearchCV instead of GridSearchCV

In [89]:
from sklearn.model_selection import RandomizedSearchCV

# Initialize LogisticRegression
lr_country = LogisticRegression(max_iter=20000, multi_class='ovr')

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'C': np.logspace(-3, 3, 7),  # Exponential range for C (e.g., 0.001, 0.01, 0.1, 1, 10, 100, 1000)
    'solver': ['liblinear', 'saga'],  # Solvers for logistic regression
    'max_iter': [1000, 10000, 20000],  # Number of iterations for convergence
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# Update n_iter to match the number of combinations
random_search = RandomizedSearchCV(estimator=lr_country, 
                                   param_distributions=param_dist, 
                                   n_iter=100,  # Set n_iter to the number of parameter combinations
                                   cv=5, 
                                   n_jobs=-1, 
                                   verbose=2, 
                                   scoring=custom_scorer, 
                                   random_state=42)

# Fit the model with the scaled training data
random_search.fit(X_train_scaled, y_train_country)

# Output the best parameters and the best F1 score
print("Best Parameters:", random_search.best_params_)
print("Best F1 Score:", random_search.best_score_)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 42 is smaller than n_iter=100. Running 42 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages

Best Parameters: {'solver': 'liblinear', 'max_iter': 1000, 'C': 1000.0}
Best F1 Score: 0.6786559537318885


In [91]:
# Best parameters found during the grid search
best_params = random_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_country = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=best_params['max_iter'],
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Group the data by 'Country Name'
sub_region_groups = opera.groupby('sub-region')

# Loop through each country group to train and predict separately
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['country_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_country.unique()) > 1:
            # Train the model on the training data
            lr_country.fit(X_train_scaled, y_train_country)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_country.predict(X_test_scaled)

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping country: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping country: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_country_growth'] = predicted_growth_values

# Sort and get the top countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted_lr2 = df_opera_country_growth_unique.head(10)
print(top_countries_predicted_lr2)

             Country Name predicted_country_growth
38935          Uzbekistan                Very High
12044       United States                Very High
29712          Kazakhstan                Very High
11613      United Kingdom                Very High
29640               Italy                Very High
11255  Russian Federation                Very High
11280              Sweden                Very High
29902         Netherlands                Very High
11319            Slovenia                Very High
29866              Mexico                Very High


In [93]:
lr_metrics_country2 = evaluate_model_performance(lr_country, X_country, y_country, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Logistic Regression Metrics Across Sub-Regions for Countries using RandomizedSearch:")
for metric, value in lr_metrics_country2.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Random Forest Metrics Across Sub-Regions for Countries using RandomizedSearch:
   accuracy: 0.7520
   precision: 0.7485
   recall: 0.7520


In [95]:
# Initialize LogisticRegression
lr_city = LogisticRegression(max_iter=20000, multi_class='ovr')

# Define the hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'C': np.logspace(-3, 3, 7),  # Exponential range for C (e.g., 0.001, 0.01, 0.1, 1, 10, 100, 1000)
    'solver': ['liblinear', 'saga'],  # Solvers for logistic regression
    'max_iter': [1000, 10000, 20000],  # Number of iterations for convergence
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# RandomizedSearchCV with custom scoring
random_search = RandomizedSearchCV(estimator=lr_city, 
                                   param_distributions=param_dist, 
                                   n_iter=100,  # Number of random parameter combinations to try
                                   cv=5,  # Number of folds in cross-validation
                                   n_jobs=-1,  # Use all CPU cores
                                   verbose=2,  # Print progress messages
                                   scoring=custom_scorer,  # Use F1 score for evaluation
                                   random_state=42)  # For reproducibility

# Fit the model with the scaled training data
random_search.fit(X_train_scaled, y_train_city)

# Output the best parameters and the best F1 score
print("Best Parameters:", random_search.best_params_)
print("Best F1 Score:", random_search.best_score_)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 42 is smaller than n_iter=100. Running 42 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 42 candidates, totalling 210 fits


/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max

Best Parameters: {'solver': 'liblinear', 'max_iter': 1000, 'C': 0.1}
Best F1 Score: 0.3973721239033546


In [96]:
# Best parameters found during the grid search
best_params = random_search.best_params_

# Initialize LogisticRegression with the best parameters
lr_city = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=best_params['max_iter'],
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Loop through each sub-region group to train and predict separately for cities
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this sub-region group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_city.unique()) > 1:
            # Train the model on the training data
            lr_city.fit(X_train_scaled, y_train_city)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_city.predict(X_test_scaled)

            # Ensure the indices of predictions and X_test match
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping sub-region: {city_name} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping sub-region: {city_name} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_city_growth'] = predicted_growth_values

# Sort and get the top cities based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 cities with highest predicted growth
top_cities_predicted = df_opera_city_growth_unique.head(10)
print(top_cities_predicted)

      iso             city predicted_city_growth
38935  uz         Tashkent             Very High
13716  au           Sydney             Very High
13410  us     White Plains             Very High
13422  us       Wilmington             Very High
33126  au        Melbourne             Very High
33004  at         Salzburg             Very High
32940  at             Graz             Very High
32984  at             Linz             Very High
32959  at        Innsbruck             Very High
13407  us  West Palm Beach             Very High


In [99]:
lr_metrics_city2 = evaluate_model_performance(lr_city, X_city, y_city, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Logistic Regression Metrics Across Sub-Regions for Cities using RandomizedSearch:")
for metric, value in lr_metrics_city2.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Logistic Regression Metrics Across Sub-Regions for Cities using RandomizedSearch:
   accuracy: 0.4754
   precision: 0.5147
   recall: 0.4754


Random Forest Classifier since the models for countries and cities were not very strong with logistic regression and might work better with RCF since it's a stronger model.

In [102]:
from sklearn.ensemble import RandomForestClassifier

# Hyperparameter distributions for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(100, 1001, 100),  # Randomly sample between 100 and 500 estimators
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],  # Can sample between 'sqrt' and 'log2'
    'bootstrap': [True, False]
}

# Initialize RandomForestClassifier
rf_country = RandomForestClassifier()

# RandomizedSearchCV with custom scoring (no n_iter)
random_search_rf = RandomizedSearchCV(estimator=rf_country, 
                                   param_distributions=param_dist,  # Param distributions for RandomizedSearchCV
                                   n_iter=100,  # Number of random combinations to sample
                                   cv=5, 
                                   n_jobs=-1, 
                                   verbose=2, 
                                   random_state=42)

# Fit the randomized search
random_search_rf.fit(X_train_scaled, y_train_country)

# Output the best parameters and the best F1 score
print("Best Parameters:", random_search_rf.best_params_)
print("Best F1 Score:", random_search_rf.best_score_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 20, 'bootstrap': False}
Best F1 Score: 0.997500699650175


In [104]:
# Best parameters found during the grid search
best_params = random_search_rf.best_params_

# Initialize RandomForestClassifier with the best parameters
rf_country = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
    random_state=42
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Initialize StandardScaler (to ensure consistent scaling)
scaler = StandardScaler()

# Loop through each country group to train and predict separately for cities
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['country_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_country.unique()) > 1:
            # Train the Random Forest Classifier with best_params
            rf_country.fit(X_train_scaled, y_train_country)  # ✅ Now fitting the model

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = rf_country.predict(X_test_scaled)  # ✅ Using the fitted model

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping the sub-region: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping the sub-region: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_country_growth'] = predicted_growth_values
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted_rf = df_opera_country_growth_unique.head(10)
print(top_countries_predicted_rf)

             Country Name predicted_country_growth
23865              Serbia                Very High
16544               Italy                Very High
9586              Hungary                Very High
9984                Japan                Very High
9273               France                Very High
10788  Russian Federation                Very High
10545              Poland                Very High
28721             Finland                Very High
10173              Latvia                Very High
28700               Spain                Very High


In [106]:
rf_metrics_country = evaluate_model_performance(rf_country, X_country, y_country, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Random Forest Metrics Across Sub-Regions for Countries")
for metric, value in rf_metrics_country.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Random Forest Metrics Across Sub-Regions for Countries
   accuracy: 0.9631
   precision: 0.9754
   recall: 0.9631


In [110]:
# Hyperparameter distributions for RandomizedSearchCV
param_dist = {
    'n_estimators': np.arange(100, 1001, 100),  # Randomly sample between 100 and 500 estimators
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],  # Can sample between 'sqrt' and 'log2'
    'bootstrap': [True, False]
}

# Initialize RandomForestClassifier
rf_city = RandomForestClassifier()

# RandomizedSearchCV with custom scoring (no n_iter)
random_search_rfy = RandomizedSearchCV(estimator=rf_city, 
                                   param_distributions=param_dist,  # Param distributions for RandomizedSearchCV
                                   n_iter=100,  # Number of random combinations to sample
                                   cv=5, 
                                   n_jobs=-1, 
                                   verbose=2, 
                                   random_state=42)

# Fit the randomized search
random_search_rfy.fit(X_train_scaled, y_train_city)

# Output the best parameters and the best F1 score
print("Best Parameters:", random_search_rfy.best_params_)
print("Best F1 Score:", random_search_rfy.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=  11.0s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=900; total time=  20.1s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=600; total time=  12.5s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=500; total time=  10.4s
[CV] END bootstrap=False, max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   9.4s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=  30.2s
[CV] END bootstrap=True, max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators

In [118]:
# Best parameters found during the grid search
best_params = random_search_rfy.best_params_

# Initialize RandomForestClassifier with the best parameters
rf_city = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
    random_state=42
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Initialize StandardScaler (to ensure consistent scaling)
scaler = StandardScaler()

# Loop through each country group to train and predict separately for cities
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_city.unique()) > 1:
            # Train the Random Forest Classifier with best_params
            rf_city.fit(X_train_scaled, y_train_city)  # ✅ Now fitting the model

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = rf_city.predict(X_test_scaled)  # ✅ Using the fitted model

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping the sub-region: {sub_region} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping the sub-region: {sub_region} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_city_growth'] = predicted_growth_values
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 countries with highest predicted growth
top_cities_predicted_rf = df_opera_city_growth_unique.head(10)
print(top_cities_predicted_rf)

      iso        city predicted_city_growth
24422  se   Stockholm             Very High
17156  no        Oslo             Very High
17090  nl     Hengelo             Very High
17085  nl   Heemstede             Very High
33145  au      Sydney             Very High
33130  au   Melbourne             Very High
8120   de    Hannover             Very High
8138   de  Heidelberg             Very High
8145   de  Heidenheim             Very High
28290  de      Passau             Very High


In [120]:
rf_metrics_city = evaluate_model_performance(rf_city, X_city, y_city, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average Random Forest Metrics Across Sub-Regions for Cities:")
for metric, value in rf_metrics_city.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average Random Forest Metrics Across Sub-Regions for Cities:
   accuracy: 0.8821
   precision: 0.8975
   recall: 0.8821


Random Forest was much stronger at predicting with accuracy and precision. I'm going to try one more model, the XGBoost.

In [125]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.9 MB/s eta 0:00:00


In [ ]:
pip install --upgrade xgboost

In [164]:
def evaluate_model_performance_1(model, X, y, sub_region_groups, scaler, bins, labels):
    """
    Trains and evaluates an XGBoost model for each sub-region, then returns the average accuracy, precision, and recall.

    Parameters:
    - model: The XGBoost model to train (e.g., XGBClassifier).
    - X: Feature DataFrame.
    - y: Target variable DataFrame.
    - sub_region_groups: Grouped data by sub-region.
    - scaler: StandardScaler instance.
    - bins: Bins for categorizing target values.
    - labels: Labels corresponding to bins.

    Returns:
    - Dictionary with average accuracy, precision, and recall.
    """
    accuracies, precisions, recalls = [], [], []

    for sub_region, group in sub_region_groups:
        if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
            # Extract features and target
            X_group = group[X.columns]
            y_group = group[y.name]

            # Convert target variable to categorical labels (Ensure labels start from 0)
            y_group_classified = pd.cut(y_group, bins=bins, labels=range(len(labels)))
            y_group_classified = y_group_classified.astype(int) - y_group_classified.min()

            # Train-test split
            X_train, X_test, y_train, y_test = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

            # Scale the training and test data
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # Handle missing classes (Ensure XGBoost knows all possible labels)
            unique_classes = set(y_train.unique())
            all_classes = set(range(len(labels)))
            missing_classes = all_classes - unique_classes

            if missing_classes:
                for missing_class in missing_classes:
                    dummy_X = X_train_scaled.mean(axis=0) + np.random.normal(0, 0.01, X_train_scaled.shape[1])
                    dummy_X = dummy_X.reshape(1, -1)
                    dummy_y = np.array([missing_class])

                    X_train_scaled = np.vstack([X_train_scaled, dummy_X])
                    y_train = np.append(y_train, dummy_y)

            # Train only if there are multiple classes
            if len(np.unique(y_train)) > 1:
                model.fit(X_train_scaled, y_train)
                predictions = model.predict(X_test_scaled)

                # Store evaluation metrics
                accuracies.append(accuracy_score(y_test, predictions))
                precisions.append(precision_score(y_test, predictions, average='weighted', zero_division=0))
                recalls.append(recall_score(y_test, predictions, average='weighted', zero_division=0))

    # Compute the average metrics
    avg_metrics_1 = {
        'accuracy': np.mean(accuracies) if accuracies else 0,
        'precision': np.mean(precisions) if precisions else 0,
        'recall': np.mean(recalls) if recalls else 0
    }

    return avg_metrics_1

In [137]:
# Independent variables (predictors)
X_country = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 
                'Year', 'Month', 'Day', 'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_country = opera['country_change_from_previous_season']

# Compute quartiles for binning
q1 = y_country.quantile(0.25)
q2 = y_country.quantile(0.50)
q3 = y_country.quantile(0.75)

# Define bins and numeric labels
bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = [0, 1, 2, 3]  # Low = 0, Medium = 1, High = 2, Very High = 3

# Convert to categorical numeric labels
y_country_classified = pd.cut(y_country, bins=bins, labels=labels).astype(int)

# Train-test split
X_train, X_test, y_train_country, y_test_country = train_test_split(X_country, y_country_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [146]:
import xgboost as xgb

# Define the hyperparameter search space
param_dist = {
    'n_estimators': np.arange(50, 500, 50),  # Number of trees
    'learning_rate': np.linspace(0.01, 0.3, 10),  # Learning rate
    'max_depth': np.arange(3, 10, 1),  # Tree depth
    'min_child_weight': np.arange(1, 10, 2),  # Minimum sum of instance weight needed in a child
    'gamma': np.linspace(0, 0.5, 5),  # Minimum loss reduction to make further partition
    'subsample': np.linspace(0.5, 1.0, 5),  # Fraction of samples used per tree
    'colsample_bytree': np.linspace(0.5, 1.0, 5),  # Fraction of features used per tree
    'reg_alpha': np.logspace(-3, 1, 5),  # L1 regularization
    'reg_lambda': np.logspace(-3, 1, 5),  # L2 regularization
}

# Initialize XGBoost classifier
xgb_country = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=42)

# Define a custom scoring function (F1-score)
custom_scorer = make_scorer(f1_score, average='weighted')

# Perform Randomized Search
random_search_xgb = RandomizedSearchCV(
    estimator=xgb_country,
    param_distributions=param_dist,
    n_iter=100,  # Number of random samples
    cv=5,  # 5-fold cross-validation
    scoring=custom_scorer,
    n_jobs=-1,  # Use all available processors
    verbose=2,
    random_state=42
)

# Fit the Randomized Search on training data (for country)
random_search_xgb.fit(X_train_scaled, y_train_country)

# Output the best parameters and best F1 score
print("Best Parameters:", random_search_xgb.best_params_)
print("Best F1 Score:", random_search_xgb.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [09:45:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [09:45:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [09:45:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [09:45:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i

Best Parameters: {'subsample': 0.875, 'reg_lambda': 1.0, 'reg_alpha': 0.01, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.1711111111111111, 'gamma': 0.25, 'colsample_bytree': 0.5}
Best F1 Score: 0.9966299707264021


In [156]:
# Best parameters found during the grid search
best_params = random_search_xgb.best_params_

# Initialize XGBoost with the best parameters
xgb_country = xgb.XGBClassifier(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    random_state=42
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Initialize StandardScaler (to ensure consistent scaling)
scaler = StandardScaler()

# Loop through each sub-region to train and predict separately
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
        # Extract features and target
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['country_change_from_previous_season']

        # Convert the target to categorical labels (Ensuring labels start from 0)
        y_group_classified = pd.cut(y_group, bins=bins, labels=range(len(labels)))
        
        # Convert to integer type (XGBoost requires integer labels)
        y_group_classified = y_group_classified.astype(int)

        # Split into train-test sets (80-20 split)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)  # Use same scaler for test data

        # Ensure all possible classes are included in training, even if missing
        if len(y_train_country.unique()) > 1:
            missing_classes = set(range(len(labels))) - set(y_train_country.unique())

            if missing_classes:
                for missing_class in missing_classes:
                    dummy_X = X_train_scaled.mean(axis=0) + np.random.normal(0, 0.01, X_train_scaled.shape[1])  # Add noise
                    dummy_X = dummy_X.reshape(1, -1)  
                    dummy_y = np.array([missing_class])

                    X_train_scaled = np.vstack([X_train_scaled, dummy_X])
                    y_train_country = np.append(y_train_country, dummy_y)

            # Train the model
            xgb_country.fit(X_train_scaled, y_train_country)

            # Make predictions
            predictions = xgb_country.predict(X_test_scaled)

            # Assign predictions to correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions
        else:
            print(f"Skipping {sub_region} due to insufficient class variation.")
            continue
    else:
        print(f"Skipping {sub_region} due to insufficient data.")
        continue

# Fill NaN values in case of skipped regions
opera['predicted_country_growth'] = predicted_growth_values.fillna("No Prediction")

# Convert predictions to numeric where possible, keeping "No Prediction" as-is
opera['numeric_growth'] = pd.to_numeric(opera['predicted_country_growth'], errors='coerce')

# Sort using numeric values (NaNs get pushed to the bottom)
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth', 'numeric_growth']].sort_values(by='numeric_growth', ascending=False)

# Drop temp numeric column after sorting
df_country_growth_sorted = df_country_growth_sorted.drop(columns='numeric_growth')

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted_xgb = df_opera_country_growth_unique.head(10)
print(top_countries_predicted_xgb)

Skipping Northern Africa due to insufficient class variation.
Skipping South-eastern Asia due to insufficient class variation.
             Country Name predicted_country_growth
25440       United States                        3
8326              Germany                        3
9015                Spain                        3
23093               Italy                        3
26600             Austria                        3
15662              Sweden                        3
11103  Russian Federation                        3
11929      United Kingdom                        3
11456             Ukraine                        3
29232             Hungary                        3


In [166]:
xgb_metrics_country = evaluate_model_performance_1(xgb_country, X_country, y_country, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average XGBoost Metrics Across Sub-Regions for Countries")
for metric, value in xgb_metrics_country.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average XGBoost Metrics Across Sub-Regions for Countries
   accuracy: 0.9910
   precision: 0.9899
   recall: 0.9910


In [168]:
# Independent variables (predictors)
X_city = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 
                'Year', 'Month', 'Day', 'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Compute quartiles for binning
q1 = y_city.quantile(0.25)
q2 = y_city.quantile(0.50)
q3 = y_city.quantile(0.75)

# Define bins and numeric labels
bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = [0, 1, 2, 3]  # Low = 0, Medium = 1, High = 2, Very High = 3

# Convert to categorical numeric labels
y_city_classified = pd.cut(y_city, bins=bins, labels=labels).astype(int)

# Train-test split
X_train, X_test, y_train_city, y_test_city = train_test_split(X_city, y_city_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [170]:
# Define the hyperparameter search space
param_dist = {
    'n_estimators': np.arange(50, 500, 50),  # Number of trees
    'learning_rate': np.linspace(0.01, 0.3, 10),  # Learning rate
    'max_depth': np.arange(3, 10, 1),  # Tree depth
    'min_child_weight': np.arange(1, 10, 2),  # Minimum sum of instance weight needed in a child
    'gamma': np.linspace(0, 0.5, 5),  # Minimum loss reduction to make further partition
    'subsample': np.linspace(0.5, 1.0, 5),  # Fraction of samples used per tree
    'colsample_bytree': np.linspace(0.5, 1.0, 5),  # Fraction of features used per tree
    'reg_alpha': np.logspace(-3, 1, 5),  # L1 regularization
    'reg_lambda': np.logspace(-3, 1, 5),  # L2 regularization
}

# Initialize XGBoost classifier
xgb_city = xgb.XGBClassifier(objective='multi:softmax', num_class=3, random_state=42, use_label_encoder=)

# Define a custom scoring function (F1-score)
custom_scorer = make_scorer(f1_score, average='weighted')

# Perform Randomized Search
random_search_xgby = RandomizedSearchCV(
    estimator=xgb_city,
    param_distributions=param_dist,
    n_iter=100,  # Number of random samples
    cv=5,  # 5-fold cross-validation
    scoring=custom_scorer,
    n_jobs=-1,  # Use all available processors
    verbose=2,
    random_state=42
)

# Fit the Randomized Search on training data (for country)
random_search_xgby.fit(X_train_scaled, y_train_city)

# Output the best parameters and best F1 score
print("Best Parameters:", random_search_xgby.best_params_)
print("Best F1 Score:", random_search_xgby.best_score_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [11:33:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [11:33:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [11:33:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/anaconda3/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [11:33:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i

Best Parameters: {'subsample': 0.875, 'reg_lambda': 0.01, 'reg_alpha': 0.1, 'n_estimators': 300, 'min_child_weight': 1, 'max_depth': 8, 'learning_rate': 0.3, 'gamma': 0.125, 'colsample_bytree': 0.625}
Best F1 Score: 0.9225022006766685


In [174]:
# Best parameters found during the grid search
best_params = random_search_xgb.best_params_

# Initialize XGBoost with the best parameters
xgb_city = xgb.XGBClassifier(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_child_weight=best_params['min_child_weight'],
    gamma=best_params['gamma'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'],
    random_state=42
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
sub_region_groups = opera.groupby('sub-region')

# Initialize StandardScaler (to ensure consistent scaling)
scaler = StandardScaler()

# Loop through each sub-region to train and predict separately
for sub_region, group in sub_region_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract features and target
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (Ensuring labels start from 0)
        y_group_classified = pd.cut(y_group, bins=bins, labels=range(len(labels)))
        
        # Convert to integer type (XGBoost requires integer labels)
        y_group_classified = y_group_classified.astype(int)

        # Split into train-test sets (80-20 split)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)  # Use same scaler for test data

        # Ensure all possible classes are included in training, even if missing
        if len(y_train_city.unique()) > 1:
            missing_classes = set(range(len(labels))) - set(y_train_city.unique())

            if missing_classes:
                for missing_class in missing_classes:
                    dummy_X = X_train_scaled.mean(axis=0) + np.random.normal(0, 0.01, X_train_scaled.shape[1])  # Add noise
                    dummy_X = dummy_X.reshape(1, -1)  
                    dummy_y = np.array([missing_class])

                    X_train_scaled = np.vstack([X_train_scaled, dummy_X])
                    y_train_city = np.append(y_train_city, dummy_y)

            # Train the model
            xgb_city.fit(X_train_scaled, y_train_city)

            # Make predictions
            predictions = xgb_city.predict(X_test_scaled)

            # Assign predictions to correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions
        else:
            print(f"Skipping {sub_region} due to insufficient class variation.")
            continue
    else:
        print(f"Skipping {sub_region} due to insufficient data.")
        continue

# Fill NaN values in case of skipped regions
opera['predicted_city_growth'] = predicted_growth_values.fillna("No Prediction")

# Convert predictions to numeric where possible, keeping "No Prediction" as-is
opera['numeric_growth'] = pd.to_numeric(opera['predicted_city_growth'], errors='coerce')

# Sort using numeric values (NaNs get pushed to the bottom)
df_city_growth_sorted = opera[['iso', 'city', 'predicted_country_growth', 'numeric_growth']].sort_values(by='numeric_growth', ascending=False)

# Drop temp numeric column after sorting
df_city_growth_sorted = df_city_growth_sorted.drop(columns='numeric_growth')

# Drop duplicates and keep the row with the highest growth for each country
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 countries with highest predicted growth
top_cities_predicted_xgb = df_opera_city_growth_unique.head(10)
print(top_cities_predicted_xgb)

      iso        city predicted_country_growth
16399  hu      Szeged                        2
7566   de      Berlin                        3
8566   de  Regensburg                        3
7515   de     Bautzen                        3
32198  us   Nashville                        0
27905  de  Heidelberg                        3
27897  de    Hannover                        3
15065  de        Kiel                        1
32292  us     Norfolk                        0
7749   de     Detmold                        3


In [176]:
xgb_metrics_city = evaluate_model_performance_1(xgb_city, X_city, y_city, sub_region_groups, scaler, bins, labels)

print(f"\n🔹 Average XGBoost Metrics Across Sub-Regions for Countries")
for metric, value in xgb_metrics_city.items():
    print(f"   {metric}: {value:.4f}")


🔹 Average XGBoost Metrics Across Sub-Regions for Countries
   accuracy: 0.8745
   precision: 0.8916
   recall: 0.8745


In [182]:
# These variables are already defined in your environment
metric_vars = [
    "lin_metrics_country", "lr_metrics_country1", "lr_metrics_country2", 
    "rf_metrics_country", "xgb_metrics_country", "lin_metrics_city", 
    "lr_metrics_city1", "lr_metrics_city2", "rf_metrics_city", "xgb_metrics_city"
]

# Create a dictionary to store the metric values
metrics_data = {}

# Iterate through each metric variable and retrieve its values dynamically
for var in metric_vars:
    if var in globals():  # Check if the variable exists in the global scope
        metrics_data[var] = globals()[var]  # Retrieve its value

# Convert to DataFrame with NaN where values are missing
final_df = pd.DataFrame(metrics_data).T

# Reorder columns for clarity
final_df = final_df[["MAE", "RMSE", "R2", "accuracy", "precision", "recall"]]

# Display the final structured DataFrame
print(final_df.head(10))

                           MAE        RMSE        R2  accuracy  precision  \
lin_metrics_country  62.530724  103.077740 -1.504155       NaN        NaN   
lr_metrics_country1        NaN         NaN       NaN  0.720633   0.716794   
lr_metrics_country2        NaN         NaN       NaN  0.752030   0.748506   
rf_metrics_country         NaN         NaN       NaN  0.963110   0.975393   
xgb_metrics_country        NaN         NaN       NaN  0.991004   0.989886   
lin_metrics_city     12.795362   19.067106 -4.087927       NaN        NaN   
lr_metrics_city1           NaN         NaN       NaN  0.520271   0.541099   
lr_metrics_city2           NaN         NaN       NaN  0.475384   0.514653   
rf_metrics_city            NaN         NaN       NaN  0.882146   0.897475   
xgb_metrics_city           NaN         NaN       NaN  0.874506   0.891568   

                       recall  
lin_metrics_country       NaN  
lr_metrics_country1  0.720633  
lr_metrics_country2  0.752030  
rf_metrics_country   0.9